<a href="https://colab.research.google.com/github/venkateshchettiar/SpamFilter/blob/master/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import numpy as np
import os
import re
import nltk
import nltk.corpus

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download("averaged_perceptron_tagger")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
file = pd.read_csv("/content/drive/My Drive/DeepLearning/train.csv")

In [ ]:
file.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [ ]:
file.shape

(1306122, 3)

In [ ]:
file['target'].value_counts()

0    1225312
1      80810
Name: target, dtype: int64

In [ ]:
file.drop(columns='qid', inplace=True)

In [ ]:
file.head()

,question_text,target
0,How did Quebec nationalists see their province...,0
1,"Do you have an adopted dog, how would you enco...",0
2,Why does velocity affect time? Does velocity a...,0
3,How did Otto von Guericke used the Magdeburg h...,0
4,Can I convert montra helicon D to a mountain b...,0


In [ ]:
file['char_count'] = file['question_text'].str.len()
file[['question_text','char_count']].head()

,question_text,char_count
0,How did Quebec nationalists see their province...,72
1,"Do you have an adopted dog, how would you enco...",81
2,Why does velocity affect time? Does velocity a...,67
3,How did Otto von Guericke used the Magdeburg h...,57
4,Can I convert montra helicon D to a mountain b...,77


### The above table contians length of characters in a sentence including spaces.

In [ ]:
file['word_count'] = file['question_text'].apply(lambda x:len(str(x).split(' ')))

In [ ]:
file[['question_text','word_count']].head()

,question_text,word_count
0,How did Quebec nationalists see their province...,13
1,"Do you have an adopted dog, how would you enco...",16
2,Why does velocity affect time? Does velocity a...,10
3,How did Otto von Guericke used the Magdeburg h...,9
4,Can I convert montra helicon D to a mountain b...,15


### The above table contians length of words in a sentence.

In [ ]:
file['#Tags'] = file['question_text'].apply(lambda x:len([x for x in x.split() if x.startswith('#')]))

In [ ]:
file[['question_text','#Tags']].head()

,question_text,#Tags
0,How did Quebec nationalists see their province...,0
1,"Do you have an adopted dog, how would you enco...",0
2,Why does velocity affect time? Does velocity a...,0
3,How did Otto von Guericke used the Magdeburg h...,0
4,Can I convert montra helicon D to a mountain b...,0


### **The above table contians numbers of '#' in a sentence.**

In [ ]:
file['#Tags'].value_counts()

0     1305794
1         307
2          18
3           2
10          1
Name: #Tags, dtype: int64

### **Count of "#' in a sentence**

In [ ]:
file['numeric'] = file['question_text'].apply(lambda x:len([x for x in x.split() if x.isdigit()]))

In [ ]:
file[['question_text','numeric']].head()

,question_text,numeric
0,How did Quebec nationalists see their province...,0
1,"Do you have an adopted dog, how would you enco...",0
2,Why does velocity affect time? Does velocity a...,0
3,How did Otto von Guericke used the Magdeburg h...,0
4,Can I convert montra helicon D to a mountain b...,0


### **The above table contians numbers of 'Numericals' in a sentence**

In [ ]:
file['numeric'].value_counts()

0     1236526
1       58281
2        9286
3        1509
4         366
5         106
6          26
7          10
10          3
9           3
8           3
15          1
13          1
11          1
Name: numeric, dtype: int64

### **Count of 'Numericals' in a sentence**

In [ ]:
max(file['numeric']),min(file['numeric'])

(15, 0)

In [ ]:
common_words = pd.Series(' '.join(file['question_text']).split()).value_counts()[:10]
common_words 

the     653948
What    417802
to      403183
a       402682
in      363131
is      331647
of      330825
I       306261
How     261930
and     251973
dtype: int64

### **similar words in a sentence**

In [ ]:
rare_words = pd.Series(' '.join(file['question_text']).split()).value_counts()[-10:]
rare_words

tournamets          1
downswing?          1
non-genital         1
Vikings/Norsemen    1
x=-y                1
norm)?              1
*(a-b)              1
easy-to-hide        1
Gurewitch           1
60days,             1
dtype: int64

### **rare words in a sentence.**

In [ ]:
def tokenize(words):
  return(word_tokenize(words))

def to_lowercase(words):
  new_words = []
  for word in words:
    new_word = word.lower()
    new_words.append(new_word)
  return new_words

def remove_punctuation(words):
  new_words = []
  for word in words:
    new_word = re.sub(r'[^\w\s]', '',word)
    if new_word != '':
      new_words.append(new_word)
  return new_words

def remove_duplicates(words):
  new_words = []
  for word in words:
    if word not in new_words:
      new_words.append(word)
  return new_words

def remove_digits(words):
  new_words = []
  for word in words:
    new_word = re.sub(r'[0-9]', "",word)
    if new_word != "":
      new_words.append(new_word)
  return new_words

def remove_stopwords(words):
  new_words = []
  a = set(stopwords.words('english'))
  for word in words:
    if word not in a:
      new_words.append(word)
  return new_words    

def lemmatizer(words):
  new_words = []
  for word in words:
    lemmatizer = WordNetLemmatizer()
    new_word = lemmatizer.lemmatize(word,wordnet.VERB)
    new_words.append(new_word)
  return new_words 

def pos_tag(words):
  new_words = []
  for token in words:
    a = nltk.pos_tag([token])
    new_words.append(a)
  return new_words    


In [ ]:
def normalize(words):
  words = tokenize(words)
  words = to_lowercase(words)
  words = remove_punctuation(words)
  words = remove_digits(words)
  words = remove_stopwords(words)
  words = lemmatizer(words)
  words = remove_duplicates(words)
  return words

In [ ]:
a=normalize(r"What is [math]\frac{\int_{1x^5}^{3x^{-5}} \tan(\tan({\boxed{\int_{1x^0}^{1x^2} \sum_{\varpi=1}^{\infty} \int_{2x^{-3}}^{2x^2} \sum_{\alpha=7}^{\infty} \underbrace{\sqrt[2]{1x^5}}_{\text{Gauss's Law of Theoretical Probability.}} d\tau dx}}^{1x^0})) d\mu}{\int_{2x^{-3}}^{1x^5} \cos(\int_{2x^2}^{1x^{-3}} \frac{\sqrt[2]{\overbrace{\underbrace{\frac{3x^3+3x^5}{\sqrt[3]{2x^{-3}}}}_{\text{Gauss's Law of Theoretical Probability.}} \times \overbrace{\tan(2x^0)}^{\text{Gauss's Law of Theoretical Probability.}}-\sum_{4=7}^{\infty} \boxed{3x^{-5}}}^{\text{Inverse Function.}}}}{{\boxed{\int_{2x^2}^{2x^4} 3x^1 d9} \div \sum_{6=6}^{\infty} \sqrt[3]{2x^2}+\sqrt[4]{\sin(2x^0+3x^0)}}^{2x^{-4}}+\boxed{\frac{\vec{\boxed{\sum_{\gamma=10}^{\infty} 1x^{-5}}}}{\frac{\sum_{\iota=2}^{\infty} 1x^{-5}-\frac{3x^{-1}}{1x^{-4}}}{\sin(\tan(3x^{-2}))}}}} \times \boxed{\sqrt[2]{{{{\sqrt[5]{2x^5}}^{2x^{-1}}}^{2x^{-1}} \div \sum_{\chi=6}^{\infty} \int_{1x^4}^{2x^{-4}} 3x^2 d\vartheta+{2x^{-3}}^{2x^{-5}}}^{3x^{-4}}}} d\mu) d\iota}[/math]?")
len(a)

35

In [ ]:
len(normalize("Studying is #hard. Sometimes you won’t be #prepared. But taking #prescription #drugs is a #dangerous way & get an #edge. Some people have gotten hooked on these #medicines & even died. Just #budget the time to hunker down and hit those #books.?"))

21

In [ ]:
sample = pd.DataFrame(file['question_text'].head())
sample

,question_text
0,How did Quebec nationalists see their province...
1,"Do you have an adopted dog, how would you enco..."
2,Why does velocity affect time? Does velocity a...
3,How did Otto von Guericke used the Magdeburg h...
4,Can I convert montra helicon D to a mountain b...


In [ ]:
sample = sample['question_text'].apply(lambda x: ' '.join(normalize(x)))
sample

0              quebec nationalists see province nation
1                adopt dog would encourage people shop
2                  velocity affect time space geometry
3          otto von guericke use magdeburg hemispheres
4    convert montra helicon mountain bike change tyres
Name: question_text, dtype: object

In [ ]:
file['Question_Text_New'] = file['question_text'].apply(lambda x:" ".join(normalize(x)))

In [ ]:
file['Question_Text_New'].head()

0              quebec nationalists see province nation
1                adopt dog would encourage people shop
2                  velocity affect time space geometry
3          otto von guericke use magdeburg hemispheres
4    convert montra helicon mountain bike change tyres
Name: Question_Text_New, dtype: object

In [ ]:
file[["Question_Text_New","target"]].head()

,Question_Text_New,target
0,quebec nationalists see province nation,0
1,adopt dog would encourage people shop,0
2,velocity affect time space geometry,0
3,otto von guericke use magdeburg hemispheres,0
4,convert montra helicon mountain bike change tyres,0


In [ ]:
file[['Question_Text_New','target']].to_csv("SpamFilter_Cleaned_Data.csv", index=False)

In [ ]:
x = pd.read_csv("/content/SpamFilter_Cleaned_Data.csv")

In [ ]:
x.head()

,Question_Text_New,target
0,quebec nationalists see province nation,0
1,adopt dog would encourage people shop,0
2,velocity affect time space geometry,0
3,otto von guericke use magdeburg hemispheres,0
4,convert montra helicon mountain bike change tyres,0
